### In this notebook we use tensorflow CNN model to find waldo.

In [57]:
%%capture

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Lambda, Resizing
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image

import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

from img_gen import create_sample

In [58]:
#constants
IMG_SIZE = 64

In [59]:
# # creating waldo images
# create_sample(IMG_SIZE,4017)

In [60]:
train_ds, test_ds = tf.keras.utils.image_dataset_from_directory(
    '../data/selfmade',
    validation_split=0.2,
    subset="both",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32)

print(test_ds.class_names)

Found 9354 files belonging to 2 classes.
Using 7484 files for training.
Using 1870 files for validation.
['notwaldo', 'waldo']


In [61]:
from collections import Counter

label_counter = Counter()

for images, labels in test_ds:
    labels_np = labels.numpy()
    label_counter.update(labels_np)

# Print the number of elements for each class
for label, count in label_counter.items():
    print(f"Class {label}: {count} elements")

Class 0: 1104 elements
Class 1: 766 elements


2024-06-13 22:51:16.633629: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [62]:
def get_conv(input_shape=(IMG_SIZE, IMG_SIZE, 3), filename=None):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


model = get_conv()

In [63]:
from tensorflow.keras.applications import ResNet50

# model = get_conv()
# model.summary()
# model2 = Sequential()
# model2.add(Resizing(224,224))
# model2.add(ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3)))
# model2.add(Flatten())
# model2.add(Dense(1024, activation='relu'))
# model2.add(Dense(2, activation='softmax'))

In [64]:
# model.compile(optimizer='adam',
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#               metrics=['accuracy'])
# 
# history = model.fit(train_ds, validation_data=test_ds, epochs=5)
# model.save('1306_1816.keras')

model = tf.keras.models.load_model('../models/1816.keras')
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 12, 12, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1024)           │     2,360,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 2)              │         2,050 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,256,072 (27.68 MB)

 Trainable params: 2,418,690 (9.23 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 4,837,382 (18.45 MB)

In [65]:
# plt.plot(history.history['accuracy'], label='accuracy')
# plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.ylim([0.3, 1])
# plt.legend(loc='lower right')


In [66]:
train_images, train_labels = next(iter(train_ds.unbatch().batch(len(train_ds))))
test_images, test_labels = next(iter(test_ds.unbatch().batch(len(test_ds))))

for type, images, labels in [['train', train_images, train_labels], ['test', test_images, test_labels]]:
    pred = model.predict(images)
    pred_labels = tf.argmax(pred, axis=1)
    pred_labels = tf.cast(pred_labels, tf.int32)

    correct = tf.reduce_sum(tf.cast(pred_labels == labels, tf.int32))
    accuracy = correct / len(labels)

    print(f"Manual checking accuracy on {type} data: {accuracy * 100:.2f}%")

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Manual checking accuracy on train data: 100.00%
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Manual checking accuracy on test data: 100.00%


In [67]:
import os

directory_path = f'../data/src1/{IMG_SIZE}'
pref = ""
filenames = [(f, 'waldo') for f in os.listdir(directory_path + '/waldo') if f.startswith(pref)] + [(f, 'notwaldo') for f
                                                                                                   in os.listdir(
        directory_path + '/notwaldo') if f.startswith(pref)]
filenames.sort()
filenames

[('10_0_0.jpg', 'notwaldo'),
 ('10_0_1.jpg', 'notwaldo'),
 ('10_0_10.jpg', 'notwaldo'),
 ('10_0_11.jpg', 'notwaldo'),
 ('10_0_12.jpg', 'notwaldo'),
 ('10_0_13.jpg', 'notwaldo'),
 ('10_0_14.jpg', 'notwaldo'),
 ('10_0_15.jpg', 'notwaldo'),
 ('10_0_2.jpg', 'notwaldo'),
 ('10_0_3.jpg', 'notwaldo'),
 ('10_0_4.jpg', 'notwaldo'),
 ('10_0_5.jpg', 'notwaldo'),
 ('10_0_6.jpg', 'notwaldo'),
 ('10_0_7.jpg', 'notwaldo'),
 ('10_0_8.jpg', 'notwaldo'),
 ('10_0_9.jpg', 'notwaldo'),
 ('10_10_0.jpg', 'notwaldo'),
 ('10_10_1.jpg', 'notwaldo'),
 ('10_10_10.jpg', 'notwaldo'),
 ('10_10_11.jpg', 'notwaldo'),
 ('10_10_12.jpg', 'notwaldo'),
 ('10_10_13.jpg', 'notwaldo'),
 ('10_10_14.jpg', 'notwaldo'),
 ('10_10_15.jpg', 'notwaldo'),
 ('10_10_2.jpg', 'notwaldo'),
 ('10_10_3.jpg', 'notwaldo'),
 ('10_10_4.jpg', 'notwaldo'),
 ('10_10_5.jpg', 'notwaldo'),
 ('10_10_6.jpg', 'notwaldo'),
 ('10_10_7.jpg', 'notwaldo'),
 ('10_10_8.jpg', 'notwaldo'),
 ('10_10_9.jpg', 'notwaldo'),
 ('10_11_0.jpg', 'notwaldo'),
 ('10_11_1.jpg

In [68]:
correct = 0
incorrect = 0
incorrect_files, correct_files = [], []

for fname, true in tqdm(filenames):
    img = image.load_img(f'../data/src1/{IMG_SIZE}/{true}/{fname}', target_size=(IMG_SIZE, IMG_SIZE, 3))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array, verbose=0)
    predicted_class = 'waldo' if prediction[0][0] < prediction[0][1] else 'notwaldo'

    if predicted_class == true:
        correct += 1
        correct_files += [f'../data/src1/{IMG_SIZE}/{true}/{fname}']
    else:
        incorrect_files += [f'../data/src1/{IMG_SIZE}/{true}/{fname}']
        incorrect += 1

print(correct, incorrect)

100%|██████████| 5376/5376 [03:11<00:00, 28.06it/s]

5345 31


In [72]:
print(correct/(correct + incorrect))

0.9942336309523809


In [69]:
counter = 0
for f in correct_files:
    if f.startswith(f'../data/src1/{IMG_SIZE}/waldo'):
        counter += 1
counter2 = 0
for f in incorrect_files:
    if f.startswith(f'../data/src1/{IMG_SIZE}/waldo'):
        counter2 += 1

print("Waldo correct: ", counter, " incorrect: ", counter2)

counter = 0
for f in correct_files:
    if f.startswith(f'../data/src1/{IMG_SIZE}/notwaldo'):
        counter += 1
counter2 = 0
for f in incorrect_files:
    if f.startswith(f'../data/src1/{IMG_SIZE}/notwaldo'):
        counter2 += 1

print("Notwaldo correct: ", counter, " incorrect: ", counter2) 

Waldo correct:  9  incorrect:  30
Notwaldo correct:  5336  incorrect:  1
